In [1]:
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow import keras

In [21]:
import os
import pandas as pd
import numpy as np

In [3]:
def get_images(directory: str, label: str) -> list:
    """return all images in a directory as a list"""
    return [(os.path.join(directory,item), label) for item in os.listdir(directory)]

In [34]:
def get_images_without_label(directory: str) -> list:
    """return all test images in a directory as a list"""
    return [os.path.join(directory, item) for item in os.listdir(directory)]

In [4]:
def create_dataframe(data: list) -> pd.DataFrame:
    """create images from a list and return a dataframe"""
    df = pd.DataFrame()
    image = []
    label = []
    for item in data:
        image.append(plt.imread(item[0]))
        label.append(item[1])
    df["image"] = image
    df["label"] = label
    
    return df

In [35]:
def create_dataframe_without_label(data: list) -> pd.DataFrame:
    """create images from a list and return a dataframe"""
    df = pd.DataFrame()
    image = []
    for item in data:
        image.append(plt.imread(item))
    df["image"] = image
    
    return df

In [36]:
train_clean_folder = "../images/train/clean/"
train_messy_folder = "../images/train/messy/"

test_folder = "../images/test/"

val_clean_folder = "../images/val/clean/"
val_messy_folder = "../images/val/messy/"

In [6]:
tem = get_images(train_clean_folder,"clean")

In [7]:
cle = create_dataframe(tem)

In [8]:
mes = get_images(train_messy_folder, "messy")
mes_df = create_dataframe(mes)

In [9]:
mes_df = mes_df.set_index(pd.Index(range(96,192)))

In [10]:
train_df = pd.concat([cle, mes_df])

In [11]:
train_df["label"].value_counts()

clean    96
messy    96
Name: label, dtype: int64

### DEFINE MODEL ARCHITECTURE

In [12]:
from tensorflow.keras import layers

In [13]:
model = keras.Sequential()

model.add(layers.Conv2D(64, (3,3), activation="relu", input_shape=(299,299,3)))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Conv2D(64, (3,3), activation="relu"))
model.add(layers.Flatten())
model.add(layers.Dense(16, activation="relu"))
model.add(layers.Dense(1, activation="sigmoid"))

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 297, 297, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 148, 148, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 146, 146, 64)      36928     
_________________________________________________________________
flatten (Flatten)            (None, 1364224)           0         
_________________________________________________________________
dense (Dense)                (None, 16)                21827600  
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 21,866,337
Trainable params: 21,866,337
Non-trainable params: 0
____________________________________________

In [15]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192 entries, 0 to 191
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   image   192 non-null    object
 1   label   192 non-null    object
dtypes: object(2)
memory usage: 3.1+ KB


In [19]:
train_df["image"][0].shape

(299, 299, 3)

In [25]:
train_df = train_df.sample(frac=1, random_state=1)

In [26]:
X = np.array([item for item in train_df["image"]])
X.shape

(192, 299, 299, 3)

In [24]:
from tensorflow.keras.utils import to_categorical

In [30]:
Y = train_df["label"].map(lambda x: 1 if x == "clean" else 0)

In [37]:
sam_t = get_images_without_label(test_folder)

test_df = create_dataframe_without_label(sam_t)

In [40]:
val_clean_list = get_images(val_clean_folder, "clean")
val_messy_list = get_images(val_messy_folder, "messy")

val_clean_df = create_dataframe(val_clean_list)
val_messy_df = create_dataframe(val_messy_list).set_index(pd.Index(range(10,20)))

In [43]:
validation_df = pd.concat([val_clean_df, val_messy_df]).sample(frac=1, random_state=1)

In [47]:
val_data = np.array([item for item in validation_df["image"]])

In [48]:
val_test = validation_df["label"].map(lambda x: 1 if x == "clean" else 0)

In [50]:
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuray"]
)

In [52]:
v